In [1]:
#TODO: Move this to utils.
import os

openai_key = os.getenv('OPENAI_API_KEY')

if openai_key:
    print("OpenAI key accessed.")
else:
    print("API Key not found. Check your environment variables.")

OpenAI key accessed.


In [2]:
import fitz  # PyMuPDF
from pathlib import Path

def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    page_texts = []
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()
        page_texts.append(text)
    return page_texts

pdf=Path("docs/QGenda Whitepaper.pdf")

raw_text = extract_text_from_pdf(pdf)
print(raw_text)


['Improving Capacity and Revenue \nthrough Effective Room Management\nHealthcare Leaders Experiencing Exam Room \nUtilization Challenges, Leading to Revenue Loss\nW H I T E PA P E R\nCopyright © 2024 QGenda, LLC All rights reserved.\n', 'Copyright © 2024 QGenda, LLC All rights reserved.\nAs healthcare leaders reimagine patient access and care delivery in a post-pandemic world, there is an\nemerging story around the need to optimize clinic exam room utilization. Better utilization can improve\noperating efficiency, patient satisfaction, and revenue capture. The 2020 Porter Research study of\n100 health system executive leaders identified the many challenges faced today with exam room\nscheduling, future expectations for optimizing exam rooms, and the impact that proper exam room\nscheduling can have on a health system’s P&L.\nIncreasing Scrutiny on Clinic Exam Rooms\nWith the limited funds available for health system capital expenditures, executives must optimize their \nexisting physic

In [3]:
# Pagination
import re
import json

# Regular expression pattern to match copyright text
pattern = r"Copyright © 2024 QGenda, LLC All rights reserved."

# Split data after each occurrence of the pattern
pages = []
current_page = []
for index, item in enumerate(raw_text):
    current_page.append(item)
    if re.search(pattern, item):
        pages.append({"index": len(pages), "text": "".join(current_page)})
        current_page = []

# Convert the pages list to JSON format
pages_json = json.dumps(pages, indent=4)

# Output the result
print(pages_json)

[
    {
        "index": 0,
        "text": "Improving Capacity and Revenue \nthrough Effective Room Management\nHealthcare Leaders Experiencing Exam Room \nUtilization Challenges, Leading to Revenue Loss\nW H I T E PA P E R\nCopyright \u00a9 2024 QGenda, LLC All rights reserved.\n"
    },
    {
        "index": 1,
        "text": "Copyright \u00a9 2024 QGenda, LLC All rights reserved.\nAs healthcare leaders reimagine patient access and care delivery in a post-pandemic world, there is an\nemerging story around the need to optimize clinic exam room utilization. Better utilization can improve\noperating efficiency, patient satisfaction, and revenue capture. The 2020 Porter Research study of\n100 health system executive leaders identified the many challenges faced today with exam room\nscheduling, future expectations for optimizing exam rooms, and the impact that proper exam room\nscheduling can have on a health system\u2019s P&L.\nIncreasing Scrutiny on Clinic Exam Rooms\nWith the limite

In [4]:
# Set up prompt for GPT
from langchain.prompts import PromptTemplate

template = """
Given the following extracted parts of a PDF document ("SOURCES") identify the primary text of the document (i.e. ignore text like 'Copyright © 2024' and 'n19%') and transcribe it.
Transcribe it strictly and do not add any other boiler plate text to you answer. It should just be text from the document. 
REMOVE encoding symbols for new lines (\n and \n\n), and make sure to look for this pattern even if they're touching another word. 
SOURCES:
{summaries}
=========
ANSWER:
"""
prompt = PromptTemplate(template=template, input_variables=["summaries"])

In [5]:
# Lazy way: Use LLM to clean up outputs.
from langchain_openai import OpenAI
from langchain.docstore.document import Document
from langchain.chains import LLMChain

llm = OpenAI(temperature=1.0)
cleaned_pages = []
for page in pages:
    raw_text = page['text']
    chain_qa = LLMChain(llm=llm, prompt=prompt)
    llm_results = chain_qa.invoke({"summaries": raw_text}, return_only_outputs=True)
    cleaned_text = llm_results['text']
    cleaned_pages.append({"index": page['index'], "text": cleaned_text})

# Convert the cleaned pages list to JSON format
cleaned_pages_json = json.dumps(cleaned_pages, indent=4)

# Output the result
print(cleaned_pages_json)

C:\Users\trott\AppData\Local\Temp\ipykernel_23388\2941083774.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_qa = LLMChain(llm=llm, prompt=prompt)


[
    {
        "index": 0,
        "text": "Improving Capacity and Revenue through Effective Room Management\nHealthcare Leaders Experiencing Exam Room Utilization Challenges, Leading to Revenue Loss"
    },
    {
        "index": 1,
        "text": "As healthcare leaders reimagine patient access and care delivery in a post-pandemic world, there is an emerging story around the need to optimize clinic exam room utilization. Better utilization can improve operating efficiency, patient satisfaction, and revenue capture. The 2020 Porter Research study of 100 health system executive leaders identified the many challenges faced today with exam room scheduling, future expectations for optimizing exam rooms, and the impact that proper exam room scheduling can have on a health system\u2019s P&L. Increasing Scrutiny on Clinic Exam Rooms With the limited funds available for health system capital expenditures, executives must optimize their existing physical space while simultaneously remaining a

In [6]:
from pathlib import Path

# Map this to the first cell
original_f = Path(pdf)
new_f = original_f.with_stem(original_f.stem + '_cleaned')
json_format = new_f.with_suffix('.json')

with open(json_format, 'w', encoding='utf-8') as file:
    file.write(cleaned_pages_json)